# 봄 데이터 전처리 진행

## 함수형 코드 진행

In [1]:
# preprocessing
import numpy as np
import pandas as pd
import tqdm
from scipy import stats

# imputer
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

In [2]:
# load data
train = pd.read_csv('../data/train_yestsnona.csv')
test_data = pd.read_csv('../data/spring_test.csv')

In [3]:
# 계절별로 빼내야됨
data = train[train['mm'].isin([2, 3, 4])]

In [4]:
# 변수간의 상관관계가 있는지 확인하고자 하는 함수
def correlation_cal(data, ycol, xcol):
    df = data.copy()
    df = df.dropna(axis = 0)
    select = []
    for names in xcol:
        if df[names].dtypes in ['int64', 'float64']:
            c, p = stats.pearsonr(df[names], df[ycol])
            if p <= 0.05 and abs(c) >= 0.5:
                select.append(names)
    
    return select + [ycol]

In [5]:
# 이상치 탐색하기
def abnormal_data(data, ycol, data_type, Q1, Q2, Q3):
    df = data.copy()
    if data_type == "train":
        Q1 = np.quantile(df[ycol][df[ycol].notnull()], 0.25)
        Q2 = np.quantile(df[ycol][df[ycol].notnull()], 0.5)
        Q3 = np.quantile(df[ycol][df[ycol].notnull()], 0.75)
        IQR = Q3 - Q1
        df[ycol][(df[ycol] < (Q2 - IQR * 1.5)) | (df[ycol] > (Q2 + IQR * 1.5))] = np.nan
        return df, Q1, Q2, Q3
    
    elif data_type == "test":
        IQR = Q3 - Q1
        df[ycol][(df[ycol] < (Q2 - IQR * 1.5)) | (df[ycol] > (Q2 + IQR * 1.5))] = np.nan
        return df
    
    else:
        print("없는 데이터 타입입니다. 확인하시고 train과 test중 하나를 입력해주세요.")

In [6]:
# imputation 적용하기 : 두 가지 방법을 활용해 적용해두기
# fit한 모델을 그대로 가져와 test셋에도 똑같이 적용하기 위해 return에 포함시킨다.
def imputation_method(method, data, cols, neightbor = 2, weight = "uniform"):
    if method == "mice":
        imputer_mice = IterativeImputer(random_state=42)
        imputer_mice.fit(data[cols])

        # 데이터 변환 (array로 반환하기 때문에 필요에 맞는 형태로 변환 후 사용)
        data[cols] = pd.DataFrame(imputer_mice.transform(data[cols]), columns=data[cols].columns)
        
        # 함수와 데이터 전부 리턴해야 나중에 테스트 셋에서도 사용할 수 있다.
        return imputer_mice, data
    
    elif method == "knn":
        # KNN을 이용한 자동 대치
        imputer_KNN = KNNImputer(n_neighbors=neightbor, weights=weight)
        imputer_KNN.fit(data[cols])

        # 데이터 변환
        data[cols] = pd.DataFrame(imputer_KNN.transform(data[cols]), columns=data[cols].columns)
        
        # 함수와 데이터 전부 리턴해야 나중에 테스트 셋에서도 사용할 수 있다.
        return imputer_KNN, data
    
    else:
        # 다른 임퓨터는 없어요
        print("그 임퓨터는 지원되지 않습니다. 이름을 확인하거나 소문자로 입력 되었는지 확인해주소.")

In [7]:
# 원하는 변수의 평균값 구해두기
def mean_number(df, needs):
    mean_list = df[needs].mean()
    
    for name in tqdm.tqdm(needs):
        df[name][df[name].isna()] = mean_list[name]
    
    print(df[needs].isna().sum())
    return df, mean_list

In [8]:
# 일사, 일조량 데이터 결측값 대치
def sun_power(selectdata, wantcol, hour, sunrise, sunset, weather):
    df = selectdata.copy()
    for name in tqdm.tqdm(wantcol):
        conditions = [
            df[name].notnull(),
            ((df[hour] < df[sunrise]) | (df[hour] > df[sunset])),
            (df[sunrise] <= df[hour]) & (df[hour] <= df[sunset]) & (df[weather].isin(['C'])),
            (df[sunrise] <= df[hour]) & (df[hour] <= df[sunset]) & (~df[weather].isin(['C']))
        ]
        choices = [
            df[name], 
            0, 
            df[name][df[weather].isin(['C'])].mean(),
            df[name][~df[weather].isin(['C'])].mean()
                  ]
        df[name] = np.select(conditions, choices)
    return df, choices

In [9]:
# 강수량, 강수유무 결측값 대치
def rain_power(selectdata, wantcol, weather):
    df = selectdata.copy()
    for name in tqdm.tqdm(wantcol):
        conditions = [
            df[name].notnull(),
            (df[weather].isin(['R'])),
            (df[weather].isin(['F', 'H'])),
            (~df[weather].isin(['R', 'F', 'H'])),
        ]
        choices = [
            df[name], 
            round(df[name][df[weather].isin(['R'])].mean(), 1),
            round(df[name][df[weather].isin(['F', 'H'])].mean(), 1),
            round(df[name][~df[weather].isin(['R', 'F', 'H'])].mean(), 1)
                  ]
        df[name] = np.select(conditions, choices)
    return df, choices

## 적용 코드

In [10]:
# 기온 이슬점 습도 풍속 대치
data2, mlist = mean_number(data, ['ta', 'td', 'hm', 'ws'])

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 408.53it/s]

ta    0
td    0
hm    0
ws    0
dtype: int64


In [11]:
# 태양과 관련된 변수 대치
data3, param_s = sun_power(data2, ['si', 'ss'], 'hh', 'sunriseh', 'sunseth', 'ww')

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 70.14it/s]


In [12]:
# 비와 관련된 변수 대치
data4, param_r = rain_power(data3, ['rn', 're'], 'ww')

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 30.50it/s]


In [13]:
# 봄이므로 적설량 없다 취급
data4['sn'][data4['sn'].isna()] = 0

In [14]:
data4.columns

Index(['stn', 'year', 'ta', 'td', 'hm', 'ws', 'rn', 're', 'ww', 'ts', 'si',
       'ss', 'sn', 'mm', 'dd', 'hh', 'sunriseh', 'sunrisem', 'sunseth',
       'sunsetm'],
      dtype='object')

In [15]:
# 이상치를 결측치로 전환과 동시에 Q1, Q2, Q3 값 뽑아주기
# test 데이테에서 또한 사용해야함
data5, Q1, Q2, Q3 = abnormal_data(data4, "ts", "train", 0, 0, 0)

In [16]:
# 시간단위를 분 단위로 바꿔주기(이 코드는 한번만 돌려야지 잘 적용된다.)
data5['ss'] = round(data5['ss'] * 60)

In [17]:
# 필요없는 변수 빼기
data6 = data5[data5.columns.difference(['sunriseh', 'sunrisem', 'sunseth', 'sunsetm', 'ww', 'Unnamed: 0'])]

In [18]:
# 상관계수 절댓값 0.5이상과 p-value가 0.05이하인 변수만 선택
# select_col = correlation_cal(data6, 'ts', list(data6.columns.difference(['ts'])))

In [19]:
# imputation 진행하기
# 연도 제외시키기
tmp = data6.drop(['year'],axis = 1)

# mice, data7 = imputation_method("mice", data6, select_col)
# knn, data8 = imputation_method("knn", data6, select_col)
imputer_mice = IterativeImputer(random_state=42)
imputer_mice.fit(tmp)

# 데이터 변환 (array로 반환하기 때문에 필요에 맞는 형태로 변환 후 사용)
tmp2 = pd.DataFrame(imputer_mice.transform(tmp), columns=tmp.columns)

# reset index
data7 = data6.reset_index().drop(['index'],axis = 1)

# 데이터에 mice한 값 집어넣기
data7['ts'] = tmp2['ts']

In [20]:
# 데이터 저장
data7.to_csv('../data/spring_impute.csv', index = False)

## Test data

In [21]:
# 값 대치
for name in tqdm.tqdm(['ta', 'td', 'hm', 'ws']):
        test_data[name][test_data[name].isna()] = mlist[name]

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 281.55it/s]


In [22]:
# 태양관련 변수 대입
for name in tqdm.tqdm(['si', 'ss']):
    conditions = [
        test_data[name].notnull(),
        ((test_data['hh'] < test_data['sunriseh']) | (test_data['hh'] > test_data['sunseth'])),
        (test_data['sunriseh'] <= test_data['hh']) & (test_data['hh'] <= test_data['sunseth']) & (test_data['ww'].isin(['C'])),
        (test_data['sunriseh'] <= test_data['hh']) & (test_data['hh'] <= test_data['sunseth']) & (~test_data['ww'].isin(['C']))
    ]
    param_s[0] = test_data[name]
    test_data[name] = np.select(conditions, param_s)

100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 200.84it/s]


In [23]:
# 비 관련 변수 대입
for name in tqdm.tqdm(['rn', 're']):
    conditions = [
        test_data[name].notnull(),
        (test_data['ww'].isin(['R'])),
        (test_data['ww'].isin(['F', 'H'])),
        (~test_data['ww'].isin(['R', 'F', 'H'])),
    ]
    param_r[0] = test_data[name]
    test_data[name] = np.select(conditions, param_r)

100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 222.03it/s]


In [24]:
# 적설량 0으로 통일
# 봄이므로 적설량 없다 취급
test_data['sn'][test_data['sn'].isna()] = 0

In [25]:
# 시간단위를 분 단위로 바꿔주기(이 코드는 한번만 돌려야지 잘 적용된다.)
test_data['ss'] = round(test_data['ss'] * 60)

In [26]:
# 필요없는 변수 빼기
test_data2 = test_data[test_data.columns.difference(['sunriseh', 'sunrisem', 'sunseth', 'sunsetm', 'ww'])]

In [27]:
# 데이터 저장
test_data2.to_csv('../data/spring_test_complete.csv', index = False)